<a href="https://colab.research.google.com/github/IS5882/UU-TAX/blob/main/Sub-task%201%20Code/UU_Tax_Subtask1_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
import re
import nltk
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
import os
import gc
import gensim

import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
!pip install transformers
import transformers
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split


# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

##0.1] Training are first 2 folds test is 3rd Fold

import base64
import requests
import pandas as pd
import os
import nltk


In [ ]:
import base64
import requests
import pandas as pd
import os
import nltk

##0.2] Load pretrained model

from simpletransformers.classification import ClassificationModel
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
# define hyperparameter
'''
"use_multiprocessing":False,
"use_multiprocessing_for_evaluation":False,
Those 2 were added for the 0% red bar thing
'''
train_args ={"reprocess_input_data": True,
             "use_multiprocessing":False,
             "use_multiprocessing_for_evaluation":False,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4} 

# Create a ClassificationModel
model_en = ClassificationModel(
    "electra",
    "google/electra-base-discriminator",
    args=train_args
)



# Read Test

In [ ]:
import base64
import requests
import pandas as pd
df_en_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_set_with_labels/subtask-1/En-Subtask1-labels.tsv', sep='\t')

df_en_test=df_en_test.set_index('ID')
del df_en_test['Construction']
df_en_test

# Read Train

In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-1/en/'
folds = [pd.read_csv(os.path.join(data_url, 'En-Subtask1-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(3)]
df_en = pd.concat(folds[0:3]) 
df_en=df_en.set_index('ID')

df_en

## Read BT

In [ ]:
df_backtrans=pd.read_excel(r'../input/backtranslationv2/df_backtrans_v2.xlsx',index_col=0)
df_backtrans[['ID']] = df_backtrans[['ID']].astype(int)
df_backtrans[['Labels']] = df_backtrans[['Labels']].astype(int)
df_backtrans=df_backtrans.set_index(['ID'])

df_backtrans


In [ ]:
df_it_train = df_backtrans[["IT-EN","Labels"]]
df_it_train=df_it_train.rename(columns={"IT-EN": "Sentence"})
df_it_train


In [ ]:
df_fr_train = df_backtrans[["FR-EN","Labels"]]
df_fr_train=df_fr_train.rename(columns={"FR-EN": "Sentence"})
df_fr_train

In [ ]:
df_bt_it_fr=pd.concat([df_it_train,df_fr_train])
df_bt_it_fr

# **NLP Aug**

In [ ]:
df_insert=pd.read_excel(r'../input/nlpaug/train_aug_ins.xlsx',index_col=0)
df_insert

In [ ]:
df_sub=pd.read_excel(r'../input/nlpaug/train_aug_sub.xlsx',index_col=0)
df_sub

# Stage1: Train on NLP aug

In [ ]:
df_nlp_auf=pd.concat([df_insert,df_sub])
df_nlp_auf

In [ ]:
df_nlp_bt=pd.concat([df_bt_it_fr,df_nlp_auf])
df_nlp_bt

In [ ]:
# Train the model
model_en.train_model(df_nlp_auf, learning_rate=3e-5)

In [ ]:
result, model_outputs, wrong_predictions = model_en.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

result

### Printing results after 1st stage

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

y_dev_pred = [int(el[1] > el[0]) for el in model_outputs]


precision_recall_fscore_support(df_en_test['Labels'],y_dev_pred , average='macro')

In [ ]:
precision_recall_fscore_support(df_en_test['Labels'],y_dev_pred , average='binary')

In [ ]:
len(model_outputs)

# 2nd Stage: Train on original training set+ Translation

In [ ]:
df_all_train_bt=pd.concat([df_en,df_it_train,df_fr_train])
df_all_train_bt

In [ ]:
df_all_train_bt=df_all_train_bt.drop_duplicates(subset='Sentence')
df_all_train_bt

In [ ]:
model_en.train_model(df_all_train_bt, learning_rate=4e-5)

In [ ]:
result2, model_outputs2, wrong_predictions2 = model_en.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

result2

In [ ]:
y_dev_pred = [int(el[1] > el[0]) for el in model_outputs2]

precision_recall_fscore_support(df_en_test['Labels'],y_dev_pred , average='macro')

In [ ]:
precision_recall_fscore_support(df_en_test['Labels'],y_dev_pred , average='binary')

In [ ]:
df_en_test[(df_en_test['Labels'] == 1) & (y_dev_pred == 1)]
df_en_testWrong2=df_en_test[(df_en_test['Labels'] != y_dev_pred)]


# test output

In [ ]:
result2, model_outputs2, wrong_predictions2 = model_en.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

In [ ]:
result2

In [ ]:
y_dev_pred_real = [int(el[1] > el[0]) for el in model_outputs2]
df_en_test['Labels']=y_dev_pred_real
df_en_test

In [ ]:
results_tsv_real=df_en_test_real.to_csv('answer_with_sent_aug1.tsv', sep="\t")